In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
historical_costs = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/data/raw/power_plants_costs/historical_costs/historical_data_for_costs.csv')

In [4]:
# Remove columns with no data, and source for analysis
historical_costs = historical_costs[historical_costs.columns[~historical_costs.columns.str.contains('Unnamed:')]].drop('Source',axis=1)
historical_costs = historical_costs.dropna(how='all')
# historical_costs = historical_costs.drop(historical_costs.loc[:,"2018":"2022"].head(0).columns, axis=1)
historical_costs

,Technology,Country,Year,Units,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,USD/MWh - 5% discount rate,NaN,NaN,NaN,NaN,NaN,80.620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.260000,NaN,NaN
1,Coal,UK,2013,USD/MWh - 5% discount rate,173.04,100.52,78.140000,NaN,NaN,87.760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,USD/MWh - 5% discount rate,120.93,65.42,73.480000,NaN,NaN,89.180000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.880000,NaN,NaN
3,Onshore,UK,2016,USD/kWh - 7.5% discount rate,NaN,NaN,0.230000,0.240000,0.230000,0.210000,0.200000,0.200000,0.19000,0.190000,0.180000,0.160000,0.15000,0.130000,0.110000,0.120000,0.120000,0.130000,0.120000,0.120000,0.110000,0.120000,0.110000,0.090000,0.080000,0.080000,0.080000,0.080000,0.080000,0.080000,NaN
4,Onshore,UK,2016,USD/MWh - 7.5% discount rate,NaN,NaN,234.265791,238.452286,225.247339,209.610418,202.227328,201.994283,188.37211,191.800909,177.292193,161.375506,152.24639,132.804218,114.072269,117.517566,121.074925,134.083202,120.392212,120.466508,114.158809,118.093503,112.663503,87.534354,84.902193,81.457285,82.946837,81.527251,77.672426,78.878602,NaN
5,PV - Utility,UK,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.453000,0.440000,0.503000,0.232000,0.200000,0.164000,0.163000,0.150000
6,PV - Utility,UK,2016,USD/MWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,453.300000,440.000000,503.200000,232.300000,199.900000,164.000000,162.800000,149.900000
7,Hydropower <10MW,Europe,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.110000,0.110000,0.110000,0.110000,0.110000,0.110000,0.110000,0.110000
8,Hydropower <10MW,Europe,2016,USD/MWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.751841,106.751841,106.751841,106.751841,106.751841,106.751841,106.751841,106.751841
9,Hydropower >10MW,Europe,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.140000,0.140000,0.140000,0.140000,0.140000,0.140000,0.140000,0.140000


In [5]:
# UK Inflation
inflation = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/data/raw/uk_economy_data/CZBH-061118.csv')
inflation = inflation.drop(inflation.index[[0,1,2,3,4]])
inflation = inflation.rename(index = str, columns = {"Title":"Year","RPI All Items: Percentage change over 12 months: Jan 1987=100":"Inflation"})
inflation.Year = pd.to_numeric(inflation.Year)
inflation.Inflation = 1+pd.to_numeric(inflation.Inflation)/100

inflation.head()
# Conversion of all prices to 2014 prices as per the BEIS Electricity Generation Cost Report


,Year,Inflation
5,1949,1.028
6,1950,1.031
7,1951,1.091
8,1952,1.092
9,1953,1.031


In [6]:
def inflation_multiplier(x):
#     print '-----------'
    current_year = int(x.Year)
    required_year = 2014
    
    return_array = []
    
    for price in x.loc['1981':'2017']:        
        if(required_year>current_year):
            inbetween_years = range(current_year+1, required_year+1)
            for i in inbetween_years:
                infl_value = inflation[inflation.Year==i].Inflation
                price = price * infl_value[0]
            return_array.append(price)
        else:
            inbetween_years = range(current_year,required_year,-1)
            for i in inbetween_years:
                infl_value = inflation[inflation.Year==i].Inflation
                price = price / infl_value[0]
            return_array.append(price)
    return_array = pd.Series(return_array)
    ret = pd.Series(x.loc['Technology':'Units'])
    return ret.append(return_array)
        
    
def non_static_inflation(price_row):
    REQUIRED_YEAR=2014
    USD_GBP_2014_RATE = 1.3036
    x=price_row[4:]
    
    
    axes = x.axes[0]
    return_list = []
    for price, year in zip(x,axes):
        year=int(year)
        if not np.isnan(price):
            if year<=REQUIRED_YEAR:
                for year_of_inflation in range(year+1,REQUIRED_YEAR+1):
                    price = price*inflation[inflation['Year']==year_of_inflation].Inflation[0]
                price_row.loc[str(year)]=price*USD_GBP_2014_RATE

            else:
                for year_of_inflation in range(year-1,2014-1,-1):
                    price = price/inflation[inflation['Year']==year_of_inflation].Inflation[0]
                price_row.loc[str(year)]=price*USD_GBP_2014_RATE
                
    # Convert prices from GBP to USD
#     price_row*USD_GBP_2014_RATE
    
    return pd.Series(price_row)

def remove_kWh(df):
    df = df[~df.Units.str.contains('kWh')]
    return df




In [7]:
hist_cost = historical_costs.apply(lambda x: inflation_multiplier(x), axis=1)
hist_cost.columns = historical_costs.columns
# offshore_price = non_static_inflation(historical_costs.iloc[-1])

# hist_cost = hist_cost.append(offshore_price, ignore_index=True)
hist_cost

,Technology,Country,Year,Units,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,USD/MWh - 5% discount rate,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,UK,2013,USD/MWh - 5% discount rate,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,USD/MWh - 5% discount rate,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,UK,2016,USD/kWh - 7.5% discount rate,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,UK,2016,USD/MWh - 7.5% discount rate,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,UK,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,UK,2016,USD/MWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,Europe,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,Europe,2016,USD/MWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,Europe,2016,USD/kWh - 7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [8]:
hist_cost[['Units']] = hist_cost['Units'].str.split("-").str[1]
hist_cost

,Technology,Country,Year,Units,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,5% discount rate,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,UK,2013,5% discount rate,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,5% discount rate,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,UK,2016,7.5% discount rate,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,UK,2016,7.5% discount rate,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,UK,2016,7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,UK,2016,7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,Europe,2016,7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,Europe,2016,7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,Europe,2016,7.5% discount rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [9]:
hist_cost['Units'] = pd.to_numeric(hist_cost['Units'].str.split("%").str[0])/100
hist_cost

,Technology,Country,Year,Units,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,0.050,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,UK,2013,0.050,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,0.050,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,UK,2016,0.075,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,UK,2016,0.075,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [10]:
hist_cost = hist_cost.rename(index=str, columns={"Units":"Discount_rate"})
hist_cost

,Technology,Country,Year,Discount_rate,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,0.050,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,UK,2013,0.050,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,0.050,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,UK,2016,0.075,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,UK,2016,0.075,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [11]:
hist_cost.drop(["Country","Year"], axis=1)

,Technology,Discount_rate,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,0.050,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,0.050,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,0.050,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,0.075,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,0.075,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [12]:
hist_cost.to_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/data/processed/power_plants/power_plant_costs/historical_power_plant_costs/historical_lcoe_costs.csv')

# Convert Historical Power Plant LCOE Costs to Long Format for Easy Formatting

In [13]:
hist_cost

,Technology,Country,Year,Discount_rate,1981,1984,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,CCGT,UK,2013,0.050,NaN,NaN,NaN,NaN,NaN,82.554880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.690240,NaN,NaN
1,Coal,UK,2013,0.050,177.19296,102.93248,80.015360,NaN,NaN,89.866240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,UK,2013,0.050,123.83232,66.99008,75.243520,NaN,NaN,91.320320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.821120,NaN,NaN
3,Onshore,UK,2016,0.075,NaN,NaN,0.223696,0.233422,0.223696,0.204244,0.194518,0.194518,0.184793,0.184793,0.175067,0.155615,0.145889,0.126437,0.106985,0.116711,0.116711,0.126437,0.116711,0.116711,0.106985,0.116711,0.106985,0.087533,0.077807,0.077807,0.077807,0.077807,0.077807,0.077807,NaN
4,Onshore,UK,2016,0.075,NaN,NaN,227.845116,231.916869,219.073838,203.865489,196.684752,196.458094,183.209273,186.544096,172.433030,156.952582,148.073674,129.164366,110.945816,114.296685,117.756546,130.408296,117.092544,117.164804,111.029984,114.856837,109.575661,85.135243,82.575223,79.224732,80.673459,79.292780,75.543607,76.716725,NaN
5,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.440584,0.427941,0.489214,0.225641,0.194518,0.159505,0.158533,0.145889
6,PV - Utility,UK,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,440.876111,427.940633,489.408469,225.933202,194.421210,159.505145,158.338034,145.791593
7,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985,0.106985
8,Hydropower <10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024,103.826024
9,Hydropower >10MW,Europe,2016,0.075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163,0.136163


In [17]:
long_lcoe = pd.melt(hist_cost, id_vars=['Technology','Discount_rate'], value_vars=['1981', '1984', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017'], value_name='lcoe', var_name="Year")

long_lcoe

,Technology,Discount_rate,Year,lcoe
0,CCGT,0.050,1981,NaN
1,Coal,0.050,1981,177.192960
2,Nuclear,0.050,1981,123.832320
3,Onshore,0.075,1981,NaN
4,Onshore,0.075,1981,NaN
5,PV - Utility,0.075,1981,NaN
6,PV - Utility,0.075,1981,NaN
7,Hydropower <10MW,0.075,1981,NaN
8,Hydropower <10MW,0.075,1981,NaN
9,Hydropower >10MW,0.075,1981,NaN


In [18]:
# Save long historical LCOE data to file
long_lcoe.to_csv('/Users/b1017579/Documents/PhD/Projects/10. ELECSIM/data/processed/power_plants/power_plant_costs/historical_power_plant_costs/historical_power_plant_costs_long.csv')